### Attack Dectection For SPD

In [1]:
# import libs
import numpy as np
import pandas as pd
import os
import scipy.io as io
from sklearn import preprocessing
from scipy.stats import pearsonr
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from numpy import array
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [2]:
## 计算黎曼距离
def riemann_distance(X,Y):
    X = np.reshape(X,(26,26))
    Y = np.reshape(Y,(26,26))
    gtype = 'A'
#     print(X.shape)
    if gtype == 'A':
        tmpEig = geneig(X,Y)
        tmp_dis = sum(mylog(tmpEig) ** 2)
    elif gtype == 'S':
        t = mylog(np.linalg.det(0.5 * (X + Y))) - 0.5 * (mylog(np.linalg.det(X))) + mylog(np.linalg.det(Y))
        if t == np.inf or np.isnan(t):
            eigX = np.linalg.eig(X)
            eigX = np.maximum(eigX,eps)
            eigY = np.linalg.eig(Y);
            eigY = np.maximum(eigY,eps)

            t = np.real(sum(mylog(np.linalg.eig(0.5*(X+Y))))) - 0.5 * (np.real(sum(mylog(eigX)))) + np.real(sum(mylog(eigY)))

        tmp_dis = t;
    elif gtype == 'J':
        tmp_dis = 0.5 * (np.dot(np.linalg.inv(Y),X)).trace() + 0.5 * (np.dot(np.linalg.inv(X),Y)).trace() - X.shape[0]
    elif gtype == 'L':
        tmp_dis = np.linalg.norm((logm(X) - logm(Y)),ord='fro') ** 2
    
    if tmp_dis <= 1e-15:
        dis = 0
    else:
        dis = np.sqrt(tmp_dis);
    return dis

## 计算广义特征值
def geneig(X,Y):
    w,v = np.linalg.eig(np.dot(np.linalg.inv(Y),X))
    return w

## 计算可能出现负数的对数函数
def mylog(X):
    CX = X + 0j
    return np.log(CX)

In [3]:
train_spd_X_path = '/mnt/hgfs/linuxfile/spd_train_npy_set/train_spd_X.npy'
train_spd_Y_path = '/mnt/hgfs/linuxfile/spd_train_npy_set/train_spd_Y.npy'
test_spd_X_path = '/mnt/hgfs/linuxfile/spd_test_npy_set/test_spd_X.npy'
test_spd_Y_path = '/mnt/hgfs/linuxfile/spd_test_npy_set/test_spd_Y.npy'

In [4]:
train_spd_X = np.load(train_spd_X_path)
train_spd_Y = np.load(train_spd_Y_path)
test_spd_X = np.load(test_spd_X_path)
test_spd_Y = np.load(test_spd_Y_path)

In [5]:
trainshape = train_spd_X.shape
testshape = test_spd_X.shape

In [6]:
train_spd_X_ = np.zeros(shape=(trainshape[0],trainshape[1] * trainshape[2]))
test_spd_X_ = np.zeros(shape=(testshape[0],testshape[1] * testshape[2]))

In [7]:
for i in range(trainshape[0]):
    train_spd_X_[i] = train_spd_X[i].ravel()
for i in range(testshape[0]):
    test_spd_X_[i] = test_spd_X[i].ravel()

In [8]:
train_X = train_spd_X_
train_Y = train_spd_Y
test_X = test_spd_X_
test_Y = test_spd_Y

In [27]:
Extra_model = ExtraTreesClassifier(random_state = 1).fit(train_X,train_Y)

test_Y_hat = Extra_model.predict(test_X)

print(accuracy_score(test_Y, test_Y_hat))
print(precision_score(test_Y, test_Y_hat,average='micro'))
print(recall_score(test_Y, test_Y_hat,average='micro'))
print(f1_score(test_Y, test_Y_hat,average='micro'))
print(classification_report(test_Y, test_Y_hat))

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


0.981675392670157
0.981675392670157
0.981675392670157
0.981675392670157
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99       249
         1.0       0.99      1.00      1.00       242
         2.0       1.00      1.00      1.00       250
         3.0       1.00      1.00      1.00       218
         4.0       1.00      1.00      1.00       243
         5.0       1.00      1.00      1.00       245
         6.0       0.95      0.88      0.91       247
         7.0       0.89      0.96      0.92       246
         8.0       1.00      0.98      0.99       243
         9.0       1.00      0.99      1.00       243
        10.0       1.00      1.00      1.00       248

    accuracy                           0.98      2674
   macro avg       0.98      0.98      0.98      2674
weighted avg       0.98      0.98      0.98      2674



In [28]:
RandomForest_model = RandomForestClassifier().fit(train_X,train_Y)

test_Y_hat = RandomForest_model.predict(test_X)

print(accuracy_score(test_Y, test_Y_hat))
print(precision_score(test_Y, test_Y_hat,average='micro'))
print(recall_score(test_Y, test_Y_hat,average='micro'))
print(f1_score(test_Y, test_Y_hat,average='micro'))
print(classification_report(test_Y, test_Y_hat))

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


0.9842931937172775
0.9842931937172775
0.9842931937172775
0.9842931937172775
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       249
         1.0       1.00      1.00      1.00       242
         2.0       1.00      1.00      1.00       250
         3.0       1.00      1.00      1.00       218
         4.0       1.00      1.00      1.00       243
         5.0       1.00      1.00      1.00       245
         6.0       0.96      0.88      0.92       247
         7.0       0.89      0.97      0.93       246
         8.0       1.00      1.00      1.00       243
         9.0       1.00      0.99      1.00       243
        10.0       1.00      1.00      1.00       248

    accuracy                           0.98      2674
   macro avg       0.99      0.98      0.98      2674
weighted avg       0.98      0.98      0.98      2674



In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [32]:
## 交叉验证选择合适的K值
k_range = range(1,64)
k_error = []
for k in k_range:
    print('第' + str(k) + '次检验')
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, train_X, train_Y, cv = 6, scoring='accuracy')
    k_error.append(1-scores.mean())

第1次检验


/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y

第2次检验


/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y

第3次检验


...

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y

第62次检验


/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y

第63次检验


/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y

In [33]:
k_error

[0.03860534679379324,
 0.039914002617311595,
 0.03467937932323795,
 0.03570760889885971,
 0.035240231819031576,
 0.03477285473920366,
 0.03533370723499729,
 0.03626846139465323,
 0.037764068050102906,
 0.038511871377827744,
 0.03869882220975873,
 0.03935315012151808,
 0.03935315012151808,
 0.039914002617311706,
 0.04000747803327742,
 0.039914002617311595,
 0.039540100953449286,
 0.03916619928958687,
 0.03963357636941478,
 0.039166199289586756,
 0.03944662553748357,
 0.03925967470555258,
 0.03963357636941478,
 0.03935315012151808,
 0.03916619928958698,
 0.03925967470555236,
 0.039727051785380496,
 0.03916619928958687,
 0.03916619928958687,
 0.03907272387362115,
 0.03907272387362115,
 0.038698822209758954,
 0.039540100953449286,
 0.03879229762572445,
 0.03982052720134621,
 0.039727051785380496,
 0.039914002617311706,
 0.04038137969713962,
 0.04075528136100226,
 0.040848756776967754,
 0.04094223219293325,
 0.04094223219293325,
 0.04056833052907083,
 0.041316133856795556,
 0.04112918302486

In [34]:
knn_model = KNeighborsClassifier(n_neighbors=7).fit(train_X, train_Y)

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [35]:
test_Y_hat = knn_model.predict(test_X)

In [10]:
knn_model = KNeighborsClassifier(n_neighbors=7, metric=riemann_distance).fit(train_X, train_Y)

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
test_spd_Y_hat = knn_model.predict(test_X)

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1403: ComplexWarning: Casting complex values to real discards the imaginary part
  out[i, j] = metric(X[i], Y[j], **kwds)
